In [1]:
import tweepy
import re

In [2]:
import urllib3
import urllib3.request

import csv
import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_colwidth = 1000

In [4]:
####input your credentials here
consumer_key = '4ha4rLgP6Ci6fEZtaqttGTKoA'
consumer_secret = '5ckLaCgfTdfmWM7qS9f2w05pDCSIWRCTHlm7RLnKwK9tCWIz9P'
access_token = '602145669-jHmxtsl0wSZDFeZxi81GcTzYrD87dRBhF78ip0qo'
access_token_secret = 'YFLMmVVdcN4gb4KDX3MeOjbjxoKnnsFvjKxjRGMkkEZ5D'

In [5]:
#define inputs here
input_hashtag = "giletjaune"
language = "fr"
since_date = "2018-03-07"
#n_tweets = 1

In [ ]:
####request to Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify = True)

inc = 0
for tweet in tweepy.Cursor(api.search,q="#"+input_hashtag,#count=n_tweets,    #this is a search by hashtags
                           lang=language,
                           since=since_date, tweet_mode='extended').items():
    inc = +1
    with open('tweets/tweet'+str(tweet.id)+'.json', 'w', encoding='utf8') as file:
        ####you need to create a 'tweets' folder
        tweet_json = tweet._json
        json.dump(tweet_json, file)
        

Rate limit reached. Sleeping for: 690


In [6]:
def is_a_retweet(tweet):
    #### returns a true if the tweet is a retweet, false if it's an original tweets 
    return tweet.get('retweeted_status',None) != None

In [7]:
#initialisation of databases, df files

d_users = {'user_id': [],
           'name':[],
           'screen_name':[],
           'location':[],
           'description':[],
           'url':[],
           'followers_count':[],
           'friends_count':[],
           'listed_count':[],
           'created_at':[],
           'favourites_count':[],
           'geo_enabled':[],
           'verified':[],
           'statuses_count':[],
           'lang':[],
           'contributors_enabled':[]
          }
df_users = pd.DataFrame(data=d_users)

d_tweets = {'tweet_id': [],
            'created_at': [],
            'text': [],
            'truncated': [],
            'source': [],
            'in_reply_to_status_id': [],
            'in_reply_to_user_id': [],
            'in_reply_to_screen_name': [],
            'user_id': [],
            'geo': [],
            'coordinates': [],
            'place': [],
            'contributors': [],
            'is_quote_status': [],
            'retweet_count': [],
            'favorite_count': [],
            'favorited': [],
            'retweeted': [],
            'lang': []         
           }
df_tweets = pd.DataFrame(data=d_tweets)


d_hashtags={'hashtag': [], 'tweet_id': []}
df_hashtags = pd.DataFrame(data=d_hashtags)

d_users_mentions = {'tweet_id':[], 'user_id': [], 'screen_name':[], 'name':[]}
df_users_mentions = pd.DataFrame(data=d_users_mentions)

d_retweet_users = {'user_id':[], 'original_user_id':[], 'original_tweet_id':[]}
df_retweet_users= pd.DataFrame(data=d_retweet_users)

d_influent_users = {'user_id':[], 
                    'name':[],
                    'retweet_count':[],
                    'favorite_count':[], 
                    'description':[] 
                   }

df_influent_users = pd.DataFrame(data=d_influent_users)


In [8]:
# store all hashtags related to a tweet
def store_hashtag(tweet):
    global input_hashtag
    global df_hashtags
    if tweet['entities']['hashtags']==[]:
        df_hashtags = df_hashtags.append({'hashtag': input_hashtag ,'tweet_id':tweet['id_str']}, ignore_index=True)
    else:
        for raw_hash in tweet['entities']['hashtags']:
            df_hashtags = df_hashtags.append({'hashtag': raw_hash['text'] ,'tweet_id':tweet['id_str']}, ignore_index=True)

#store all user mentions contained in the tweet
def store_user_mentions(tweet):
    global df_users_mentions
    tweet_id = tweet['id_str']
    for raw_mention in tweet['entities']['user_mentions']:
        screen_name = raw_mention['screen_name']
        name = raw_mention['name']
        user_id = raw_mention['id_str']
        df_users_mentions = df_users_mentions.append({'tweet_id':tweet_id, 'user_id': user_id, 'screen_name':screen_name, 'name':name},ignore_index=True)

#store the author of the tweet        
def store_user(tweet):
    global df_users
    user = tweet['user']
    if user['id_str'] not in df_users['user_id'].values:
        
        d_user = {'user_id': user['id_str'],
                  'name':user['name'],
                  'screen_name':user['screen_name'],
                  'location':user['location'],
                  'description':user['description'],
                  'url':user['url'],
                  'followers_count':user['followers_count'],
                  'friends_count':user['friends_count'],
                  'listed_count':user['listed_count'],
                  'created_at':user['created_at'],
                  'favourites_count':user['favourites_count'],
                  'geo_enabled':user['geo_enabled'],
                  'verified':user['verified'],
                  'statuses_count':user['statuses_count'],
                  'lang':user['lang'],
                  'contributors_enabled':user['contributors_enabled']
                 }
        df_users = df_users.append(d_user,ignore_index=True)

#store the content of the tweet
def store_tweet(tweet):
    global df_tweets
    text = re.sub(r"http\S+", "", tweet['full_text'])
    df_tweets = df_tweets.append({'tweet_id':  tweet['id_str'],
            'created_at': tweet['created_at'],
            'text': text,
            'truncated': tweet['truncated'],
            'source': tweet['source'],
            'in_reply_to_status_id': tweet['in_reply_to_status_id_str'],
            'in_reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'user_id': tweet['user']['id_str'],
            'geo': tweet['geo'],
            'coordinates': tweet['coordinates'],
            'place': tweet['place'],
            'contributors': tweet['contributors'],
            'is_quote_status': tweet['is_quote_status'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'favorited': tweet['favorited'],
            'retweeted': tweet['retweeted'],
            'lang': tweet['lang'] },ignore_index=True)

#called when the tweet is a retweet: store the original author of the tweet as well as the user who retweeted it,
# and the original id of the tweet
def store_retweet_user(tweet):
    global df_retweet_users
    if is_a_retweet(tweet):
        df_retweet_users = df_retweet_users.append({'user_id':tweet['user']['id_str'],
                                                    'original_user_id':tweet['retweeted_status']['user']['id_str'],
                                                    'original_tweet_id':tweet['retweeted_status']['id_str']}, ignore_index=True)
##Never used
##T0D0: proper encapsulation
def store_influent_users(n):
    #n is the number of influent users return
    df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).head()
    for i in range(n):
        d_influent_users = {'user_id':df_favorite_count[0], 
                            'name':[],
                            'retweet_count':[],
                            'favorite_count':[], 
                            'description':[] 
                           }
        

In [9]:
#### loading the databases

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
import os

path_to_json = 'tweets/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for json_file in json_files:
    
    with open(path_to_json+json_file) as tweet:
        tweet = json.load(tweet)
        if is_a_retweet(tweet):
            store_retweet_user(tweet)
        else:  #store tweets only when they are original tweets
            store_tweet(tweet)
            store_user(tweet)
            store_hashtag(tweet)
            store_user_mentions(tweet)
        
        
        

In [10]:
df_tweets.head(10);

In [11]:
#tweet example
df_tweets['text'].iloc[0]

'Insupportable d\'entendre le #GiletJaune en roue libre sur #LCI dire, par exemple, que l\'élection d\'#EmmanuelMacron est "une erreur de l\'Histoire", et ajouter "j\'aimerais bien qu\'il bosse au lieu de faire du ski". Sans que la journaliste ne le pousse à s\'expliquer davantage...'

In [12]:
#primary keys are ok if both return false
print(df_tweets.duplicated('tweet_id').any())
print(df_users.duplicated('user_id').any())

False
False


# Aggregations


## Top users

In [13]:
df_most_favorited_users = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_retweeted_users = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_favorited_users['favorite_count'] = df_most_favorited_users['favorite_count'].astype('int32')
df_most_retweeted_users['retweet_count'] = df_most_retweeted_users['retweet_count'].astype('int32')

df_most_favorited_users['user_id'] = df_most_favorited_users['user_id'].astype(object)
df_most_retweeted_users['user_id'] = df_most_retweeted_users['user_id'].astype(object)

In [14]:
df_most_favorited_users_with_info = df_most_favorited_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [19]:
df_most_retweeted_users_with_info.head(15);

In [14]:
df_most_favorited_users_with_info.head(15);

In [15]:
df_most_retweeted_users_with_info.reset_index()
df_most_favorited_users_with_info.reset_index();

In [20]:
print(df_most_retweeted_users_with_info['screen_name'])
print(df_most_favorited_users_with_info['screen_name'])

user_id
472852289               Brevesdepresse
1028668294751039488     GiletsJaunesFr
338955445                    tcabarrus
388972391                  AnasseKazib
2148731538                   jylgallou
3028247758                RTenfrancais
742382030                  gaetan34070
2470583048               CharlesBaudry
2653885316                  jeangarr78
859407061230252032     Circonscripti18
203894274               cecileollivier
717040304218632194     exvulgusscienta
1099297175417380865        TVPopulaire
3306815273               RevPermanente
3688356795                jpbordes4873
Name: screen_name, dtype: object
user_id
1028668294751039488     GiletsJaunesFr
338955445                    tcabarrus
472852289               Brevesdepresse
388972391                  AnasseKazib
742382030                  gaetan34070
2653885316                  jeangarr78
3028247758                RTenfrancais
2148731538                   jylgallou
804606773848133633     philippe_barthe
2470583048     

In [21]:
df_tweets_by_user = df_tweets.groupby(['user_id']).size().to_frame().reset_index()
df_most_favorited_users_with_info = df_most_favorited_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [22]:
df_most_favorited_users_with_info;

In [23]:
top_influencers_fav = df_most_favorited_users_with_info[['screen_name','description', 'followers_count', 'friends_count','favorite_count',0]]
top_influencers_rt = df_most_retweeted_users_with_info[['screen_name','description', 'followers_count', 'friends_count','retweet_count',0]]
top_influencers_fav.columns = ["Name","Description","Number of followers","Number of friends","Number of favorites","Number of tweets"]
top_influencers_rt.columns = ["Name","Description","Number of followers","Number of friends","Number of retweets","Number of tweets"]
top_influencers_fav = top_influencers_fav.reset_index().drop(columns = ["user_id"])
top_influencers_rt = top_influencers_rt.reset_index().drop(columns = ["user_id"]);
top_influencers_fav['Number of followers'] = top_influencers_fav['Number of followers'].astype(int)
top_influencers_fav['Number of friends'] = top_influencers_fav['Number of friends'].astype(int)
top_influencers_rt['Number of followers'] = top_influencers_rt['Number of followers'].astype(int)
top_influencers_rt['Number of friends'] = top_influencers_rt['Number of friends'].astype(int)

In [24]:
#add ratios
top_influencers_fav.loc[:,'Like Tweet Ratio'] = top_influencers_fav.loc[:,"Number of favorites"]/top_influencers_fav.loc[:,"Number of tweets"]
top_influencers_rt.loc[:,'Retweet Tweet Ratio'] = top_influencers_rt.loc[:,"Number of retweets"]/top_influencers_rt.loc[:,"Number of tweets"]

In [25]:
top_influencers_fav_names = top_influencers_fav['Name'].values
top_influencers_rt_names = top_influencers_rt['Name'].values
print(top_influencers_rt_names)

['Brevesdepresse' 'GiletsJaunesFr' 'tcabarrus' 'AnasseKazib' 'jylgallou'
 'RTenfrancais' 'gaetan34070' 'CharlesBaudry' 'jeangarr78'
 'Circonscripti18' 'cecileollivier' 'exvulgusscienta' 'TVPopulaire'
 'RevPermanente' 'jpbordes4873']


## Top hashtags for this input hashtags and by users

In [26]:
top_influencers_fav.style.set_properties(**{'text-align': 'left'});

In [27]:
df_tweets_hashtags = df_hashtags.set_index('tweet_id').join(df_tweets.set_index('tweet_id'))

In [28]:
df_tw_hash_user = df_tweets_hashtags.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))

In [37]:
#df_aggretated = df_tw_hash_user.groupby(['user_id','hashtag']).size().reset_index()
df_aggretated = df_tw_hash_user.groupby(['screen_name','hashtag']).size().reset_index()

In [68]:
df_aggretated.columns = ["Name", "#", "Used"]
df_aggretated;

In [43]:
#useful encapsulation, aint it?
def get_tweet_by_userid(user_id):
    df_tweets[df_tweets['user_id'] == 'user_id']

def get_tweet_by_username(name):
    df_tweets_user_info = df_tweets.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))
    df_tweets_user_info = df_tweets_user_info[df_tweets_user_info['screen_name'] == name]
    df_tweets_user_info = df_tweets_user_info[['tweet_id','text','created_at', 'favorite_count', 'retweet_count']]
    return df_tweets_user_info

def get_related_hashtags_by_username(name):
    d_name = df_aggretated[df_aggretated['Name'] == name][['#',"Used"]].sort_values("Used", ascending = False)
    return d_name

In [44]:
get_tweet_by_username('Brevesdepresse')

,tweet_id,text,created_at,favorite_count,retweet_count
user_id,,,,,
472852289,1107688525783289856,"⚡🇨🇵INFO - Un #giletjaune a perdu un œil samedi sur les Champs-Élysées suite à un tir de #LBD (BFMTV). Selon le journaliste David Dufresne, 22 personnes ont été éborgnées depuis le début de cette crise sociale.",Mon Mar 18 17:01:39 +0000 2019,598.0,1039.0


In [45]:
#output 
for name in top_influencers_rt_names:
    df_rel = get_related_hashtags_by_username(name)
    print(name)
    df_rel.to_csv('top_hashtags_by_'+name+'.csv',index=False)
    df_rel=[]

Brevesdepresse
GiletsJaunesFr
tcabarrus
AnasseKazib
jylgallou
RTenfrancais
gaetan34070
CharlesBaudry
jeangarr78
Circonscripti18
cecileollivier
exvulgusscienta
TVPopulaire
RevPermanente
jpbordes4873


In [46]:
get_related_hashtags_by_username('Brevesdepresse')

,#,Used
253,LBD,1
254,giletjaune,1


# NLP


In [47]:
import spacy

In [48]:
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import fr_core_news_sm
nlp = fr_core_news_sm.load()
#!python -m spacy download fr_core_web_lg

In [49]:
doc = nlp(df_tweets["text"][3])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [50]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [51]:
review = str(" ".join([i.lemma_ for i in doc]))

In [52]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [53]:
for i in nlp(review):
    print(i,"=>",i.pos_)

rien => PRON
que => ADV
sur => ADP
le => DET
journée => NOUN
de => ADP
samedi => NOUN
dernier => ADJ
, => PUNCT
le => DET
ville => NOUN
de => ADP
Paris => PROPN
estimer => VERB
le => DET
dégât => NOUN
à => ADP
5 => NUM
million => NOUN
d’ => PUNCT
euro => NOUN
. => PUNCT
le => DET
facture => NOUN
# => ADJ
giletjaun => ADJ
depuis => ADP
le => DET
début => NOUN
du => DET
mouvement => NOUN
en => ADP
être => AUX
désormais => ADV
à => ADP
20 => NUM
million => NOUN
. => PUNCT
@rtlfrance => NOUN


In [54]:
parser = French()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [55]:
tqdm.pandas()
df_tweets["processed_text"] = df_tweets["text"].progress_apply(spacy_tokenizer)

100%|██████████| 783/783 [00:03<00:00, 218.34it/s]


In [56]:
df_tweets.head(10)["processed_text"];

In [57]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words=stopwords, lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(df_tweets["processed_text"])

In [58]:
##How many topics do you want to find??
NUM_TOPICS = 6

In [59]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [60]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [61]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [62]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [63]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('jaune', 70.21517008669059), ('gilet', 64.9601382291061), ('giletsjaunes', 57.895210710221214), ('manifestation', 37.687601333169894), ('acte', 26.00523217951027), ('yellowvests', 25.56481410114665), ('france', 24.300112619260503), ('mars', 24.267062521895596), ('giletjaunes', 23.562862462037458), ('mois', 22.691886454665717)]
Topic 1:
[('faire', 42.17246309895365), ('giletsjaunes', 32.368270182406185), ('falloir', 31.500528820129944), ('luire', 23.586176169005135), ('prendre', 19.436856136931958), ('youtube', 17.51618493121578), ('ter', 16.300634760432093), ('yellowvests', 14.737064093360534), ('europeennes', 14.63798809462457), ('casseur', 13.929166440842124)]
Topic 2:
[('france', 109.4551396942014), ('giletsjaunes', 108.33125680838569), ('macron', 101.22277321701723), ('dictature', 82.26001186177811), ('plaire', 42.955785169317195), ('publique', 41.917712582386024), ('dan', 33.58269610985232), ('mettre', 31.574688507358594), ('pouvoir', 29.697821064504637), ('p

In [64]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('giletsjaunes', 5.00460507323224), ('yellowvests', 0.6051159039395224), ('europeennes', 0.38906624574927856), ('ric', 0.3252284877849237), ('paris', 0.2924428452944739), ('giletsjaunesparis', 0.2863501119596797), ('chalecosamarillos', 0.2650244601703079), ('jaunes', 0.25651705122716256), ('ephilippepm', 0.2092915190382774), ('partir', 0.20322022112854196)]
Topic 1:
[('faire', 3.031212657245806), ('aller', 1.1788672250421715), ('voir', 0.48758044623980257), ('falloir', 0.45836058255199447), ('ter', 0.3561119692231165), ('drouet', 0.3043416962328316), ('paris', 0.30164238728496484), ('bfmtv', 0.2852488106630809), ('devoir', 0.2706282777129441), ('prendre', 0.2571361476226955)]
Topic 2:
[('france', 2.6777456096881957), ('macron', 1.9250274888045773), ('fran', 0.6874960331742863), ('dictature', 0.5328050592789828), ('ais', 0.5247484028791614), ('lci', 0.4449420066387918), ('castaner', 0.44018711984549647), ('cnews', 0.3970802847157433), ('politique', 0.344300673297538

In [65]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('giletsjaunes', 0.6966699233513366), ('acte', 0.30091720605287037), ('france', 0.24926972524999746), ('macron', 0.2341114969237363), ('faire', 0.1693427612756951), ('dan', 0.1223174657963461), ('jaune', 0.1150603142960139), ('gilet', 0.11228857959723595), ('paris', 0.11166628946331529), ('ric', 0.09990610896842671)]
Topic 1:
[('faire', 0.4949648647187125), ('plaire', 0.3980561595853617), ('dan', 0.2537102041530325), ('aller', 0.22657238185908174), ('pouvoir', 0.176847873922892), ('voir', 0.1653747841336591), ('mettre', 0.14084506303963884), ('falloir', 0.1342757237137937), ('jaune', 0.13398586577397167), ('ter', 0.10433236200062689)]
Topic 2:
[('france', 0.5238234392854481), ('macron', 0.36829727318977484), ('fran', 0.1474863477067775), ('ais', 0.13249206696501328), ('dictature', 0.10218342618722662), ('lci', 0.09698916638066127), ('castaner', 0.08069305141975293), ('politique', 0.08051442434769006), ('cnews', 0.07468882479645657), ('acte', 0.07322465032158115)]
T

In [66]:
# Transforming an individual sentence
text = spacy_tokenizer("Les gilets jaune tous unis contre Macron. Tous dans la rue jusqu'à la démission")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.35896774 0.02112734 0.02111155 0.02088853 0.23734235 0.34056248]


In [67]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
2       9.554444 -139.420929       1        1  22.985541
3      -9.474203    4.049434       2        1  18.546468
5      23.348379  -64.594322       3        1  16.038367
1      90.250114 -100.835793       4        1  15.802268
4      78.489746  -12.166090       5        1  15.308145
0     -52.078449  -74.598267       6        1  11.319210, topic_info=    Category        Freq           Term       Total  loglift  logprob
213  Default   67.000000          jaune   67.000000  30.0000  30.0000
181  Default   60.000000          gilet   60.000000  29.0000  29.0000
163  Default  112.000000         france  112.000000  28.0000  28.0000
318  Default   55.000000         prison   55.000000  27.0000  27.0000
284  Default   54.000000          paris   54.000000  26.0000  26.0000
105  Default   65.000000      dictature   65.000000  25.0000  25.0000
52   Default   67.000000        casseur   67.000000  24.0000  24.0000
113  Default   55.000000         drouet   55.000000  23.0000  23.0000
236  Default  115.000000         macron  115.000000  22.0000  22.0000
2    Default  115.000000           acte  115.000000  21.0000  21.0000
245  Default   55.000000  manifestation   55.000000  20.0000  20.0000
54   Default   50.000000       castaner   50.000000  19.0000  19.0000
141  Default   93.000000          faire   93.000000  18.0000  18.0000
100  Default   32.000000     democratie   32.000000  17.0000  17.0000
417  Default   48.000000    yellowvests   48.000000  16.0000  16.0000
142  Default   61.000000        falloir   61.000000  15.0000  15.0000
249  Default   22.000000           mars   22.000000  14.0000  14.0000
182  Default   22.000000    giletjaunes   22.000000  13.0000  13.0000
185  Default  265.000000   giletsjaunes  265.000000  12.0000  12.0000
234  Default   25.000000          luire   25.000000  11.0000  11.0000
323  Default   34.000000       publique   34.000000  10.0000  10.0000
260  Default   28.000000           mois   28.000000   9.0000   9.0000
11   Default   27.000000            ais   27.000000   8.0000   8.0000
149  Default   20.000000         fermer   20.000000   7.0000   7.0000
403  Default   40.000000       violence   40.000000   6.0000   6.0000
135  Default   31.000000    europeennes   31.000000   5.0000   5.0000
41   Default   18.000000          black   18.000000   4.0000   4.0000
13   Default   61.000000          aller   61.000000   3.0000   3.0000
354  Default   26.000000         samedi   26.000000   2.0000   2.0000
214  Default   25.000000         jaunes   25.000000   1.0000   1.0000
..       ...         ...            ...         ...      ...      ...
149   Topic6   17.092173         fermer   20.760340   1.9842  -3.6329
402   Topic6    5.024758          ville    6.479199   1.9244  -4.8571
208   Topic6    5.436423    information    7.064253   1.9167  -4.7784
260   Topic6   20.513554           mois   28.102439   1.8639  -3.4504
258   Topic6    3.991723       mocratie    5.604529   1.8393  -5.0873
20    Topic6    4.212596         angers    5.947389   1.8338  -5.0334
46    Topic6    6.127853       bordeaux    8.824881   1.8139  -4.6586
324   Topic6    6.994561          puter   10.947168   1.7307  -4.5263
91    Topic6    4.512848         crever    7.174196   1.7151  -4.9646
245   Topic6   34.069739  manifestation   55.222641   1.6957  -2.9431
205   Topic6    3.369226            ind    5.731131   1.6474  -5.2568
98    Topic6    2.925647         demain    5.532925   1.5415  -5.3980
37    Topic6    2.929764         besoin    5.642119   1.5233  -5.3966
291   Topic6    2.361110         pendre    4.690764   1.4922  -5.6123
361   Topic6    3.002653           soci    6.210828   1.4519  -5.3720
417   Topic6   23.110692    yellowvests   48.374005   1.4400  -3.3312
219   Topic6    4.651895        justice   10.433563   1.3709  -4.9342
288   Topic6    4.022984          payer    9.482651   1.3212  -5.0795
3